In [ ]:
!pip install transformers
# from google.colab import drive
# drive.mount('/content/drive')
import pandas as pd
import sklearn
import torch # 파이토치 패키지 임포트
import torch.nn as nn # 자주 사용하는 torch.nn패키지를 별칭 nn으로 명명
from torch.utils.data import DataLoader
import pandas as pd
from torch.utils.data import Dataset # Dataset 클래스 임포트
import transformers
from transformers import AdamW, get_linear_schedule_with_warmup

     |████████████████████████████████| 3.5 MB 5.3 MB/s 
     |████████████████████████████████| 6.8 MB 28.7 MB/s 
     |████████████████████████████████| 67 kB 2.7 MB/s 
     |████████████████████████████████| 895 kB 45.8 MB/s 
     |████████████████████████████████| 596 kB 35.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
train_data = pd.read_csv("/content/drive/MyDrive/colab_data/cj_data/data/final_dataset/N2C_train_v2_tokenized.csv")
test_data = pd.read_csv("/content/drive/MyDrive/colab_data/cj_data/data/final_dataset/N2C_test_v2_tokenized.csv")

data_df = pd.concat([train_data, test_data])

# make ctg code mapping table
ctg_dict = data_df[['middle_classification', 'main_ctg', 'midd_ctg']].drop_duplicates()


In [ ]:
val_data = train_data[:1024]
train_data = train_data[1024:]

In [ ]:
test_data.midd_ctg.unique().shape[0]

165

In [ ]:

# import torch # 파이토치 패키지 임포트
# import torch.nn as nn # 자주 사용하는 torch.nn패키지를 별칭 nn으로 명명
# from transformers import BertConfig, BertModel
# # 허깅페이스의 트랜스포머 패키지에서 BertConfig, BertModel 클래스 임포트

# class CategoryClassifier(nn.Module):
#     """상품정보를 받아서 대/중/소/세 카테고리를 예측하는 모델    
#     """
#     def __init__(self, cfg):
#       super(CategoryClassifier, self).__init__()

#       # 글로벌 설정 값 사용
#       self.cfg = cfg
#       # 버트모델의 설정값을 멤버 변수로 저장
#       self.bert_cfg = BertConfig( 
#           cfg.vocab_size, # 사전 크기
#           hidden_size=cfg.hidden_size, # 히든 크기
#           num_hidden_layers=cfg.nlayers, # 레이어 층 수
#           num_attention_heads=cfg.nheads, # 어텐션 헤드의 수
#           intermediate_size=cfg.intermediate_size, # 인터미디어트 크기
#           hidden_dropout_prob=cfg.dropout, # 히든 드롭아웃 확률 값
#           attention_probs_dropout_prob=cfg.dropout, # 어텐션 드롭아웃 확률 값 
#           max_position_embeddings=cfg.seq_len, # 포지션 임베딩의 최대 길이
#           type_vocab_size=cfg.type_vocab_size, # 타입 사전 크기
#       )
#       # 텍스트 인코더로 버트모델 사용
#       self.text_encoder = BertModel(self.bert_cfg)

#       def get_classifiier(target_size):
#         return nn.Sequential(
#           nn.Linear(cfg.text_encoder_size, cfg.hidden_size),
#                     nn.LayerNorm(cfg.hidden_size),
#                     nn.Dropout(cfg.dropout),
#                     nn.ReLU(),
#                     nn.Linear(cfg.hidden_size, target_size),
#         )

#       # 대 카테고리 분류기
#       self.main_cls = get_classifiier(cfg.main_ctg)
#       # 중 카테고리 분류기
#       self.midd_cls = get_classifiier(cfg.midd_ctg)



#     def forward(self, token_ids, token_mask, token_types, label=None):
#       """        
#       매개변수
#       token_ids: 전처리된 상품명을 인덱스로 변환하여 token_ids를 만들었음
#       token_mask: 실제 token_ids의 개수만큼은 1, 나머지는 0으로 채움
#       token_types: ▁ 문자를 기준으로 서로 다른 타입의 토큰임을 타입 인덱스로 저장
#       img_feat: resnet50으로 인코딩된 이미지 피처
#       label: 정답 대/중/소/세 카테고리
#       """
#       # 전처리된 상품명을 하나의 텍스트벡터(text_vec)로 변환
#       # 반환 튜플(시퀀스 아웃풋, 풀드(pooled) 아웃풋) 중 시퀀스 아웃풋만 사용
#       text_output = self.text_encoder(token_ids, token_mask, token_type_ids=token_types)[0]
      
#       # 시퀀스 중 첫 타임스탭의 hidden state만 사용. 
#       text_vec = text_output[:, 0]




#       # 결합된 벡터로 대카테고리 확률분포 예측
#       main_pred = self.main_cls(text_vec)
#       # 결합된 벡터로 중카테고리 확률분포 예측
#       midd_pred = self.midd_cls(text_vec)


#       main_label, midd_label = label.split(1,1)

#       loss_fuc = nn.CrossEntropyLoss(ignore_index= -1)

#     #   loss_fuc.cuda()

#       main_loss = loss_fuc(main_pred, main_label.view(-1))
#       # 중카테고리의 예측된 확률분포와 정답확률 분포의 차이를 손실로 반환
#       midd_loss = loss_fuc(midd_pred, midd_label.view(-1))

#       # loss = (1*main_loss + 1.7*midd_loss) / 2

#       # 중뷴류 정확도를 높이기 위하여
#       loss = midd_loss

#       return loss, [main_pred, midd_pred]


In [ ]:
## Make DataLoader

In [ ]:
# import pandas as pd
# from torch.utils.data import Dataset # Dataset 클래스 임포트
# import re # 정규식표현식 모듈 임포트 

# class CategoryDataSet(Dataset):
#     def __init__(self, df_data, token2id, tokens_max_len=64, type_vocab_size=30):
#       """        
#       매개변수
#       df_data: 상품타이틀, 카테고리 등의 정보를 가지는 데이터프레임
#       token2id: token을 token_id로 변환하기 위한 맵핑 정보를 가진 딕셔너리
#       tokens_max_len: tokens의 최대 길이. 상품명의 tokens가 이 이상이면 잘라서 버림
#       type_vocab_size: 타입 사전의 크기
#       """
#       self.tokens = df_data['tokens'].values # 전처리된 상품명    
#     #   self.input_embedding = df_data['name_embedding'].values
#       self.tokens_max_len = tokens_max_len
#       # self.labels = df_data[['bcateid', 'mcateid', 'scateid', 'dcateid']].values
#       self.token2id = token2id 
#       self.p = re.compile('▁[^▁]+') # ▁기호를 기준으로 나누기 위한 컴파일된 정규식
#       self.type_vocab_size = type_vocab_size
#       self.labels = df_data[['main_ctg', 'midd_ctg']].values

#       #custom
#       # self.sentence_model = sentence_model

#     def __getitem__(self,idx):

#       if idx >= len(self):
#             raise StopIteration

#       # idx에 해당하는 상품명 가져오기. 상품명은 문자열로 저장돼 있음
#       tokens = self.tokens[idx]
#       if not isinstance(tokens, str):
#           tokens = ''
      
#       # 상품명을 ▁기호를 기준으로 분리하여 파이썬 리스트로 저장
#       # "▁직소퍼즐 ▁1000 조각 ▁바다 거북 의 ▁여행 ▁pl 12 75" =>
#       # ["▁직소퍼즐", "▁1000 조각", "▁바다 거북 의", "▁여행", "▁pl 12 75"]
#       tokens = self.p.findall(tokens)
      
#       # ▁ 기호 별 토큰타입 인덱스 부여
#       # ["▁직소퍼즐", "▁1000 조각", "▁바다 거북 의", "▁여행", "▁pl 12 75"] =>
#       # [     0     ,     1    1  ,    2     2  2 ,     3   ,   4  4   4 ]
#       token_types = [type_id for type_id, word in enumerate(tokens) for _ in word.split()]       
#       tokens = " ".join(tokens) # ▁기호로 분리되기 전의 원래의 tokens으로 되돌림

#       # 토큰을 토큰에 대응되는 인덱스로 변환
#       # "▁직소퍼즐 ▁1000 조각 ▁바다 거북 의 ▁여행 ▁pl 12 75" =>
#       # [2291, 784, 2179, 3540, 17334, 30827, 1114, 282, 163, 444]
#       # "▁직소퍼즐" => 2291
#       # "▁1000" => 784
#       # "조각" => 2179
#       # ...
#       token_ids = [self.token2id[tok] if tok in self.token2id else 0 for tok in tokens.split()]
      
#       # token_ids의 길이가 max_len보다 길면 잘라서 버림
#       if len(token_ids) > self.tokens_max_len:
#           token_ids = token_ids[:self.tokens_max_len]      
#           token_types = token_types[:self.tokens_max_len]
      
#       # token_ids의 길이가 max_len보다 짧으면 짧은만큼 PAD값 0 값으로 채워넣음
#       # token_ids 중 값이 있는 곳은 1, 그 외는 0으로 채운 token_mask 생성
#       token_mask = [1] * len(token_ids)
#       token_pad = [0] * (self.tokens_max_len - len(token_ids))
#       token_ids += token_pad
#       token_mask += token_pad
#       token_types += token_pad # max_len 보다 짧은만큼 PAD 추가






#       # 넘파이(numpy)나 파이썬 자료형을 파이토치의 자료형으로 변환
#       token_ids = torch.LongTensor(token_ids)
#       token_mask = torch.LongTensor(token_mask)
#       token_types = torch.LongTensor(token_types)
      
#       # token_types의 타입 인덱스의 숫자 크기가 type_vocab_size 보다 작도록 바꿈
#       token_types[token_types >= self.type_vocab_size] = self.type_vocab_size-1 


#       # 대/중/소/세 라벨 준비
#       label = self.labels[idx]
#       label = torch.LongTensor(label)
#       # label = self.labels[idx]
#       # label = torch.LongTensor(label)

#       # 크게 3가지 텍스트 입력, 이미지 입력, 라벨을 반환한다.
#       return token_ids, token_mask, token_types, label
#       # 
      
#     def __len__(self):
#       # 가지고 있는 데이터셋의 길이를 반환한다.
#       return len(self.tokens) # 1314



In [ ]:

def calc_cate_acc(pred, label):
    """
    대/중/소/세 카테고리별 정확도와 전체(overall) 정확도를 반환
    전체 정확도는 대회 평가 방법과 동일한 가중치로 계산
    """
    main_pred, midd_pred = pred    
    _, main_idx = main_pred.max(1)
    _, midd_idx = midd_pred.max(1)

    #(label[:, 0]>0) -> (label[:, 0]>=0) 후에 문제 생길 수도 잇ㅇㄹ거 같아
        
    main_acc = (main_idx == label[:, 0]).sum().item() / (label[:, 0]>=0).sum().item()
    midd_acc = (midd_idx == label[:, 1]).sum().item() / (label[:, 1]>=0).sum().item()
    
    o_acc = (main_acc + 1.7*midd_acc)/2

    return o_acc, main_acc, midd_acc

def save_checkpoint(state, model_path, model_filename, is_best=False):
    print('saving cust_model ...')
    if not os.path.exists(model_path):
        os.makedirs(model_path)
    torch.save(state, os.path.join(model_path, model_filename))
    if is_best:
        torch.save(state, os.path.join(model_path, 'best_' + model_filename))

In [ ]:
## Make main() as trainer

In [ ]:
def train(train_loader, val_loader,model, optimizer, epoch, scheduler):
  """    
    한 에폭 단위로 학습을 시킵니다.

    매개변수
    train_loader: 학습 데이터셋에서 배치(미니배치) 불러옵니다.
    model: 학습될 파라미터를 가진 딥러닝 모델
    optimizer: 파라미터를 업데이트 시키는 역할
    scheduler: learning_rate를 감소시키는 역할
  """
  model.train()

  for step, (token_ids, token_mask, token_types, label) in enumerate(train_loader):

    # to cuda
    # token_ids, token_mask, token_types, label = (token_ids.cuda(), token_mask.cuda(), token_types.cuda(), label.cuda())


    loss, pred = model(token_ids, token_mask, token_types, label)

    optimizer.zero_grad() # 옵티마이저 내의 그래디언트 초기화
    loss.backward()

    scheduler.step()    # 스케쥴러로 learning_rate 조절
    optimizer.step()    # 옵티마이저로 파라미터 업데이터

    if step % CFG.print_freq == 0 or step ==(len(train_loader) - 1):
      o_acc, main_acc, midd_acc = calc_cate_acc(pred, label)

      print(f'{step} step train acc     : {o_acc, main_acc, midd_acc}')

      if step % 1000 == 0 or step ==(len(train_loader) - 1):
        validation(model, val_loader)
  return

def validation(model, val_loader):

  for step, (token_ids, token_mask, token_types, label) in enumerate(val_loader):

    with torch.no_grad():
        loss, pred = model(token_ids, token_mask, token_types, label)
    
    o_acc, main_acc, midd_acc = calc_cate_acc(pred, label)


    print('='*100)
    print(f'{step} step val acc     : {o_acc, main_acc, midd_acc}')
    print('+'*100)
  return

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
from torch.utils.data import DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup

import sys
sys.path.append('/content/drive/MyDrive/colab_data/cj_data/model/Bert_DNN_Classification')
import category_model_from_scratch
import category_dataset_from_scratch
import random
import numpy as np

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = 'cpu'
print(device)

# Param Set
class CFG:
  epochs = 5
  batch_size = 64
  learning_rate=3.0e-4
  weight_decay=0.01
  warmup_steps = 100
  print_freq=10

  text_encoder_size = 768
  hidden_size = 768
  dropout = 0.2
  
  seed = 7

  vocab_size = 32000 # 토큰의 유니크 인덱스 개수
  type_vocab_size = 30 # 타입의 유니크 인덱스 개수
  seq_len=64 # 토큰의 최대 길이
  nlayers = 2
  nheads=8 # BERT의 head 개수
  intermediate_size=256 # TRANSFORMER셀의 intermediate 크기


  main_ctg = data_df.main_ctg.unique().shape[0]
  midd_ctg = data_df.midd_ctg.unique().shape[0]

# 랜덤 시드를 설정하여 매 코드를 실행할 때마다 동일한 결과를 얻게 합니다.
os.environ['PYTHONHASHSEED'] = str(CFG.seed)
random.seed(CFG.seed)
np.random.seed(CFG.seed)
torch.manual_seed(CFG.seed)    
torch.cuda.manual_seed(CFG.seed)
torch.backends.cudnn.deterministic = True


# 토큰을 대응되는 인덱스로 치환할 때 사용될 딕셔너리를 로딩합니다.
PROCESSED_DATA_DIR = '/content/drive/MyDrive/colab_data/cj_data/data/bert/' # 전처리된 데이터가 저장될 디렉터리
VOCAB_DIR =os.path.join(PROCESSED_DATA_DIR, 'vocab') # 전처리에 사용될 사전 파일이 저장될 디렉터리

vocab = [line.split('\t')[0] for line in open(os.path.join(VOCAB_DIR, 'spm_v2.vocab'), encoding='utf-8').readlines()]
token2id = dict([(w, i) for i, w in enumerate(vocab)])
print('loading ... done')

train_db = category_dataset_from_scratch.CategoryDataSet(train_data, token2id , CFG.seq_len, CFG.type_vocab_size)
train_loader = DataLoader(train_db, batch_size = CFG.batch_size)

val_db = category_dataset_from_scratch.CategoryDataSet(val_data, token2id , CFG.seq_len, CFG.type_vocab_size)
val_loader = DataLoader(val_db, batch_size = 1024)


# Model Set
# model = category_model.CategoryClassifier(CFG)
# model = category_model_from_scratch.CategoryClassifier(CFG)
MODEL_PATH = '/content/drive/MyDrive/colab_data/cj_data/saved_model/'
# model = model.to(device)

num_train_optimization_steps = int(
        len(train_db) / CFG.batch_size) * (CFG.epochs)
print('num_train_optimization_steps', num_train_optimization_steps)

# Optimizer Set
param_optimizer = list(model.named_parameters())   
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params':[p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 
     'weight_decay': 0.0}
]
optimizer = AdamW(model.parameters(), lr = 3.0e-4)

# Sheduler Set
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=CFG.warmup_steps,
                                            num_training_steps=num_train_optimization_steps)
print('use WarmupLinearSchedule ...')
print(model)


for epoch in range(CFG.epochs):
  print(epoch)
  train(train_loader,val_loader, model, optimizer, epoch, scheduler)

  # 모델의 파라미터를 저장합니다.
  save_checkpoint({
      'epoch': epoch + 1,
      'arch': 'transformer',
      'state_dict': model.state_dict(),
      'log': log_df,
      },
      MODEL_PATH, f'model_from_scratch_v2_batch{CFG.batch_size}_epoch{epoch+2}.pt',
  )

cpu
loading ... done
num_train_optimization_steps 35180
use WarmupLinearSchedule ...
initial learning rate:0.0
CategoryClassifier(
  (text_encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(64, 768)
      (token_type_embeddings): Embedding(30, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.2, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(i

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:247: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust

0 step train acc     : (0.665625, 0.109375, 0.71875)
0 step val acc     : (0.730078125, 0.072265625, 0.81640625)
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
10 step train acc     : (0.6734375, 0.125, 0.71875)
20 step train acc     : (0.62109375, 0.046875, 0.703125)
30 step train acc     : (0.7671875, 0.046875, 0.875)
40 step train acc     : (0.70078125, 0.046875, 0.796875)
50 step train acc     : (0.7328125, 0.03125, 0.84375)
60 step train acc     : (0.74765625, 0.140625, 0.796875)
70 step train acc     : (0.7296874999999999, 0.078125, 0.8125)
80 step train acc     : (0.771875, 0.109375, 0.84375)
90 step train acc     : (0.7640625, 0.09375, 0.84375)
100 step train acc     : (0.7609374999999999, 0.140625, 0.8125)
110 step train acc     : (0.7296874999999999, 0.078125, 0.8125)
120 step train acc     : (0.7328125, 0.03125, 0.84375)
130 step train acc     : (0.80625, 0.125, 0.875)
140 step train acc     : (0.61796875, 0.09375, 0.6718

## Test Section

In [ ]:

# Param Set
class CFG:
  epochs = 3
  batch_size = 256
  learning_rate=3.0e-4
  weight_decay=0.01
  warmup_steps = 100
  print_freq=100

  text_encoder_size = 768
  hidden_size = 768
  dropout = 0.2
  
  seed = 7

  vocab_size = 32000 # 토큰의 유니크 인덱스 개수
  type_vocab_size = 30 # 타입의 유니크 인덱스 개수
  seq_len=64 # 토큰의 최대 길이
  nlayers = 2
  nheads=8 # BERT의 head 개수
  intermediate_size=256 # TRANSFORMER셀의 intermediate 크기

  main_ctg = data_df.main_ctg.unique().shape[0]
  midd_ctg = data_df.midd_ctg.unique().shape[0]

import sys
sys.path.append('/content/drive/MyDrive/colab_data/cj_data/model/Bert_DNN_Classification')
import category_model_from_scratch
import category_dataset_from_scratch

model = category_model_from_scratch.CategoryClassifier(CFG)
mode_state_dict = torch.load('/content/drive/MyDrive/colab_data/cj_data/saved_model/model_from_scratch_v2_batch64_epoch1.pt')
model.load_state_dict(mode_state_dict['state_dict'], strict = True)

<All keys matched successfully>

In [ ]:
# Test
import torch

# 토큰을 대응되는 인덱스로 치환할 때 사용될 딕셔너리를 로딩합니다.
PROCESSED_DATA_DIR = '/content/drive/MyDrive/colab_data/cj_data/data/bert/' # 전처리된 데이터가 저장될 디렉터리
VOCAB_DIR =os.path.join(PROCESSED_DATA_DIR, 'vocab') # 전처리에 사용될 사전 파일이 저장될 디렉터리

vocab = [line.split('\t')[0] for line in open(os.path.join(VOCAB_DIR, 'spm_v2.vocab'), encoding='utf-8').readlines()]
token2id = dict([(w, i) for i, w in enumerate(vocab)])
print('loading ... done')

# Test Data Load
test_db = category_dataset_from_scratch.CategoryDataSet(test_data, token2id , CFG.seq_len, CFG.type_vocab_size)
test_loader = DataLoader(test_db, batch_size = 1)
ctg_dict = data_df[['middle_classification', 'main_ctg', 'midd_ctg']].drop_duplicates()

def inference(model, model_name):
    score = 0
    
    incorrect_pred = []
    for step, (token_ids, token_mask, token_types, label) in enumerate(test_loader):

        # with문 내에서는 그래디언트 계산을 하지 않도록 함
        with torch.no_grad():
            # model은 배치 데이터를 입력 받아서 예측 결과 및 loss 반환

        #     token_ids, token_mask, token_types, label = (
        # token_ids.cuda(), token_mask.cuda(), token_types.cuda(), label.cuda())

         loss, pred = model(token_ids, token_mask, token_types, label)

        main_label, midd_label = label[0]

        main_pred, midd_pred = pred
        _, main_idx = main_pred.max(1)
        _, midd_idx = midd_pred.max(1)

        if int(midd_label) != int(midd_idx):
            incorrect_pred.append([test_data.iloc[step, :]['name'], ctg_dict[ctg_dict.midd_ctg == int(midd_label)].middle_classification.iloc[0], ctg_dict[ctg_dict.midd_ctg == int(midd_idx)].middle_classification.iloc[0]])
            # print(midd_label, midd_idx)
            # print('='*100)
            # print(test_data.iloc[step, :])
            # print(f'label: {ctg_dict[ctg_dict.midd_ctg == int(midd_label)].middle_classification.iloc[0]}')
            # print(f'pred : {ctg_dict[ctg_dict.midd_ctg == int(midd_idx)].middle_classification.iloc[0]}')
        else:
            score += 1

        o_acc, main_acc, midd_acc = calc_cate_acc(pred, label)

        # print(o_acc, main_acc, midd_acc)
    print('='*100)
    
    print(f'model_name: {model_name} acc score: {score / len(test_loader)}')

In [ ]:
# epoch 별 성능 평가
for i in range(4):
    model = category_model_from_scratch.CategoryClassifier(CFG)
    mode_state_dict = torch.load(f'/content/drive/MyDrive/colab_data/cj_data/saved_model/model_from_scratch_v2_batch64_epoch{i+1}.pt')
    model.load_state_dict(mode_state_dict['state_dict'], strict = True)

    inference(model, f'model_from_scratch_v2_batch64_epoch{i+1}')